# ETL with Python   
## By “Aseem Shaikh"

### Executive summary : The script is designed to perform a ETL process. The task involves implementing a comprehensive logging process to record each step, including errors and warnings. The process starts by creating dedicated folders for downloaded zip file, extracted contents, logging and output. Subsequently, data is downloaded from a specified URL, and the downloaded zip file is extracted. The extracted data, comprising CSV, JSON, and XML files, is then transformed and integrated into a unified Pandas DataFrame. As part of the transformation, the price column is doubled, and the values are rounded to two decimal places. Finally, the processed and transformed data is saved as a CSV file in the designated output folder.

#### Logging to record executions

In [18]:
#Setting up logging.
import logging
from datetime import datetime
import os

base_directory = os.getcwd()

if not os.path.exists('logs'):os.mkdir('logs')
# Configure logging
logging.basicConfig(
    filename=base_directory+'\logs\logs.etl',
    filemode='a',
    format='[%(levelname)s] - %(asctime)s - %(processName)s - %(message)s',
    force=True,
    level= logging.DEBUG
    )

logging.info('ETL process Started')

#### Setting up directories

In [19]:
# Create directories if not exist

import os

directories = ['temp', 'data', 'output', 'logs']
for directory in directories:
    try:
        if not os.path.exists(directory):
           os.makedirs(directory)
           logging.info('Created Folder -'+directory)
        else:
            logging.info('Folder Exists %s', directory)
    except:
        logging.error(directory)

#### Download data from URL

In [20]:
#Downloading data from the URL

import requests

URL = 'https://elasticbeanstalk-us-east-2-340729127361.s3.us-east-2.amazonaws.com/prices.zip'
logging.info('Download Started')
try:
    response = requests.get(URL)
    response.raise_for_status()  # Check for HTTP errors
    with open(os.path.join(base_directory+'\\temp', 'prices.zip'), 'wb') as zip_file:
            zip_file.write(response.content)
    logging.info('Download Completed')
except Exception as e:
    logging.error(f'Error during download and extraction: {e}')

#### Extract Data

In [21]:
#Extacting the Data

import zipfile
logging.info('Extracting Zipfile')
try:
    with zipfile.ZipFile(base_directory+'\\temp\\prices.zip', 'r') as zip_ref:
        zip_ref.extractall(base_directory+'\data')
        logging.info('Extraction Completed')
except zipfile.BadZipFile:
    logging.error('Not a zip file or a corrupted zip file')

#### Integrate data from files into a common Pandas DataFrame.

In [22]:
#Integrate data into Pandas

import pandas as pd


file_directory = base_directory+'\data' # Specify the directory containing CSV files
zero_dataframe = pd.DataFrame() #Empty Dataframe to store all data

logging.info('Loading Data in DATAFRAME')

try:
    #CSV Files
    print(logging.info('CSV data transformation to dataframe Started'))
    files = [file for file in os.listdir(file_directory) if file.endswith('.csv')] # List all CSV files in the directory

    # Load each CSV file into Pandas DataFrame and concatenate them
    for file in files:
        file_path = os.path.join(file_directory, file)
        df = pd.read_csv(file_path)
        zero_dataframe = pd.concat([zero_dataframe, df], ignore_index=True)
    logging.info('CSV data transformation to dataframe Completed')

    #JSON FILE
    logging.info('JSON data transformation to dataframe Started')
    files = [file for file in os.listdir(file_directory) if file.endswith('.json')] # List all CSV files in the directory
    
    # Load each JSON file into Pandas DataFrame and concatenate them
    for file in files:
        file_path = os.path.join(file_directory, file)
        df = pd.read_json(file_path, lines=True)
        zero_dataframe = pd.concat([zero_dataframe, df], ignore_index=True)
    logging.info('JSON data transformation to dataframe Completed')
    

    #XML FILE
    logging.info('XML data transformation to dataframe Started')
    files = [file for file in os.listdir(file_directory) if file.endswith('.xml')] # List all CSV files in the directory
    
    # Load each XML file into Pandas DataFrame and concatenate them
    for file in files:
        file_path = os.path.join(file_directory, file)
        df = pd.read_xml(file_path)
        zero_dataframe = pd.concat([zero_dataframe, df], ignore_index=True)
    logging.info('XML data transformation to dataframe Completed')

except Exception as e:
# Display the combined DataFrame
    logging.error(e)

None


#### Double the price and round up to 2 decimal points

In [23]:
try:
    logging.info('Double the price and round up to 2 decimal points Started')
    zero_dataframe['price'] = (zero_dataframe.price*2).round(2)
    logging.info('Double the price and round up to 2 decimal points Completed')
except Exception as e:
    logging.error(e)

In [24]:
zero_dataframe

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,10000.00,Petrol
1,sx4,2013,14179.10,Diesel
2,ciaz,2017,21641.79,Petrol
3,wagon r,2011,8507.46,Petrol
4,swift,2014,13731.34,Diesel
...,...,...,...,...
85,camry,2006,7462.69,Petrol
86,land cruiser,2010,104477.61,Diesel
87,corolla altis,2012,17611.94,Petrol
88,etios liva,2013,10298.51,Petrol


#### Store dataframe to CSV

In [25]:
#Store dataframe to CSV

try:
    logging.info('Converting dataframe to CSV Started')
    zero_dataframe.to_csv(base_directory+'\output\output.csv', index=False)
    logging.info('Converting dataframe to CSV Completed')

except Exception as e:
    logging.error('Error %s',e)

logging.info('ETL process Completed')
logging.shutdown()